In [ ]:
pip install requests newspaper3k lxml_html_clean bs4 fake_useragent python-docx


In [ ]:
# The guardian
# Premium times
# The Nation


[]

In [10]:
import requests
from newspaper import Article, Config

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.request_timeout = 10
    config.browser_user_agent = user_agent
    page = Article(url, config = config)
    

    try:
        page.download()
        page.parse()
        return page.text, page.authors, page.publish_date
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'

full_content('https://www.premiumtimesng.com/regional/north-central/489977-man-42-remanded-for-allegedly-raping-11-yr-old-girl.html')

('', [], None)

In [25]:
# Read file, replace \n with comma, make each line a string
with open('/workspace/testing/premiumtimesngcom_sexual_assault_articles_revelance.txt', 'r') as file:
    content = file.read()

# Replace newlines with commas and split into strings
modified_content = content.split('\\n')
modified_content

# print(f"Found {len(strings)} strings")
# print("First 5 strings:")
# for i, string in enumerate(strings[:5], 1):
#     print(f"{i}. '{string}'")

# # If you want them as a comma-separated string:
# comma_separated = ', '.join(strings)
# print(f"\nComma-separated: {comma_separated[:200]}...")

# If you want to save the comma-separated version:
# with open('output.txt', 'w') as file:
#     file.write(comma_separated)

['https://www.premiumtimesng.com/entertainment/naija-fashion/798577-alleged-sexual-assault-speed-darlington-defers-naptip-summons-proposes-new-date.html',
 'https://www.premiumtimesng.com/foreign/world-foreign/194440-ex-diplomat-in-court-for-sexual-assault-black-magic-ritual.html',
 'https://www.premiumtimesng.com/foreign/world-foreign/266391-court-declares-bill-cosby-guilty-of-sexual-assault.html',
 'https://www.premiumtimesng.com/gender/757102-nigerian-govt-urges-universities-to-establish-sexual-assault-referral-centres.html',
 'https://www.premiumtimesng.com/gender/777551-herbalist-sentenced-to-14-years-imprisonment-for-attempted-sexual-assault.html',
 'https://www.premiumtimesng.com/gender/792715-fct-sexual-assault-centre-receives-345-sgbv-cases-in-four-years.html',
 'https://www.premiumtimesng.com/investigationspecial-reports/192813-exclusive-professor-indicted-by-court-for-sexual-assault-forgery-appointed-vice-chancellor-of-nigerian-federal-university.html',
 'https://www.premium

In [26]:
import requests
from newspaper import Article, Config
from docx import Document
import re
from datetime import datetime
import os

def clean_filename(filename):
    """Clean filename by removing invalid characters"""
    # Remove or replace invalid characters for filenames
    filename = re.sub(r'[<>:"/\\|?*]', '', filename)
    filename = re.sub(r'[^\w\s-]', '', filename)
    filename = re.sub(r'[-\s]+', '-', filename)
    return filename.strip('-')

def get_article_and_save_doc(url):
    """
    Get article content, title, date and save as Word doc
    Filename format: "Title - Date.docx"
    """
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config=config)

    try:
        print(f"Downloading article from: {url}")
        page.download()
        page.parse()
        
        # Get article details
        title = page.title if page.title else "Untitled Article"
        content = page.text if page.text else "No content retrieved"
        publish_date = page.publish_date
        authors = page.authors if page.authors else ["Unknown Author"]
        
        # Format date for filename
        if publish_date:
            date_str = publish_date.strftime("%Y-%m-%d")
        else:
            date_str = datetime.now().strftime("%Y-%m-%d")
        
        # Create filename: "Title - Date.docx"
        filename = f"{clean_filename(title)} - {date_str}.docx"
        
        # Limit filename length to avoid issues
        if len(filename) > 200:
            short_title = clean_filename(title)[:150]
            filename = f"{short_title} - {date_str}.docx"
        
        # Create Word document
        doc = Document()
        
        # Add title as heading
        doc.add_heading(title, 0)
        
        # Add metadata
        doc.add_paragraph(f"Date: {date_str}")
        doc.add_paragraph(f"Source: {url}")
        doc.add_paragraph("")  # Empty line
        
        # Add content
        # Split content into paragraphs and add each one
        paragraphs = content.split('\\n\\n')
        for paragraph in paragraphs:
            if paragraph.strip():
                doc.add_paragraph(paragraph.strip())
        
        # Save document
        folder_path = "/workspace/testing/premiun_times_rape_doc_files" # Example for Windows
        # Or: folder_path = "/home/youruser/MyWordDocuments" # Example for Linux/macOS
        

        # Ensure the folder exists (create it if it doesn't)
        os.makedirs(folder_path, exist_ok=True)

        # Construct the full path
        full_path = os.path.join(folder_path, filename)

        # Save the document to the specified path
        doc.save(full_path)
        
        
        print(f"✅ Article saved as: {filename}")
        print(f"Title: {title}")
        print(f"Date: {date_str}")
        print(f"Authors: {', '.join(authors)}")
        print(f"Content length: {len(content)} characters")
        
        return {
            'title': title,
            'date': publish_date,
            'authors': authors,
            'content': content,
            'filename': filename,
            'success': True
        }
        
    except Exception as e:
        print(f"❌ Error retrieving content from {url}: {e}")
        return {
            'error': str(e),
            'url': url,
            'success': False
        }

def process_multiple_urls(urls):
    """
    Process multiple URLs and save each as a separate doc file
    """
    results = []
    
    for i, url in enumerate(urls, 1):
        print(f"\\n--- Processing article {i}/{len(urls)} ---")
        result = get_article_and_save_doc(url)
        results.append(result)
        
        if result['success']:
            print(f"Success: {result['filename']}")
        else:
            print(f"Failed: {url}")
    
    # Summary
    successful = len([r for r in results if r['success']])
    print(f"\\n=== SUMMARY ===")
    print(f"Total articles processed: {len(urls)}")
    print(f"Successful: {successful}")
    print(f"Failed: {len(urls) - successful}")
    
    return results

# Example usage for single article
if __name__ == "__main__":
    # Single article
    # url = 'https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/'
    # result = get_article_and_save_doc(url)
    
    # Multiple articles example (uncomment to use)
    # urls = [
    #     'https://thenationonlineng.net/bbnaija-s10-19-housemates-face-possible-eviction/',
    #     'https://example.com/article2',
    #     'https://example.com/article3'
    # ]
    results = process_multiple_urls(modified_content)

\n--- Processing article 1/61 ---


✅ Article saved as: Alleged-Sexual-Assault-Speed-Darlington-defers-NAPTIP-summons-proposes-new-date - 2025-06-02.docx
Title: Alleged Sexual Assault: Speed Darlington defers NAPTIP summons, proposes new date
Date: 2025-06-02
Authors: Friday Omosola, .Wp-Block-Co-Authors-Plus-Coauthors.Is-Layout-Flow, Class, Wp-Block-Co-Authors-Plus, Display Inline, .Wp-Block-Co-Authors-Plus-Avatar, Where Img, Height Auto Max-Width, Vertical-Align Bottom .Wp-Block-Co-Authors-Plus-Coauthors.Is-Layout-Flow .Wp-Block-Co-Authors-Plus-Avatar, Vertical-Align Middle .Wp-Block-Co-Authors-Plus-Avatar Is .Alignleft .Alignright
Content length: 3036 characters
Success: Alleged-Sexual-Assault-Speed-Darlington-defers-NAPTIP-summons-proposes-new-date - 2025-06-02.docx
\n--- Processing article 2/61 ---
✅ Article saved as: Untitled-Article - 2025-09-04.docx
Title: Untitled Article
Date: 2025-09-04
Authors: Unknown Author
Content length: 20 characters
Success: Untitled-Article - 2025-09-04.docx
\n--- Processing article 3/